In [1]:
import os
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.losses import BinaryCrossentropy

import dataset
import test_lstm

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

2023-03-02 01:09:01.414849: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
lie_trial_path = './processed_lie/' #60 entries
truth_trial_path = './processed_truth/' #61 entries
MU3D_path = './processed/' # 300 entries

In [3]:
# # no split by person
X, Y = dataset.LSTM_preprocessing(truth_trial_path, lie_trial_path, numOfFrames=20)

TEST_RATIO = 0.2

xTrain, xTest = train_test_split(X, test_size=TEST_RATIO, shuffle=False)
yTrain, yTest = train_test_split(Y, test_size=TEST_RATIO, shuffle=False)

In [4]:
# split by person
# xTrain, yTrain, xTest, yTest = dataset.createDatasetLSTM(lie_trial_path, truth_trial_path, 0.2, byPerson=False)

In [5]:
LSTM_NEURONS = 32
DROPOUT = 0
EPOCHS = 10
OPTIM = 'adam'
# LOSS = 'binary_crossentropy'
LOSS = BinaryCrossentropy(from_logits=False)

n_inputRows = xTrain.shape[0] 
n_timesteps = xTrain.shape[1]
n_features = xTrain.shape[2]

def LSTM_Model(neuro, drop, loss, optim, step, feat):
  model = Sequential()
  model.add(LSTM(neuro, return_sequences=False, batch_input_shape=(None, step, feat)))
  
  model.add(Dropout(drop))
  model.add(Dense(1, activation='sigmoid')) #softmax

  model.compile(loss=loss, optimizer=optim, metrics=['accuracy'])
  return model

MODEL = LSTM_Model(LSTM_NEURONS, DROPOUT, LOSS, OPTIM, n_timesteps, n_features)
MODEL.summary()
MODEL.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=EPOCHS, verbose=1)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                5504      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 5,537
Trainable params: 5,537
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1381/1381 [==============================] - 9s 5ms/step - loss: 0.4567 - accuracy: 0.7609 - val_loss: 0.3907 - val_accuracy: 0.8041
Epoch 2/10
1381/1381 [==============================] - 7s 5ms/step - loss: 0.3782 - accuracy: 0.8064 - val_loss: 0.3687 - val_accuracy: 0.8140
Epoch 3/10
1381/1381 [========

In [6]:
prediction = MODEL.predict(xTest[0:250])
label = yTest[0:250]

for i in range(len(prediction)):
  if prediction[i] > 0.5:
    prediction[i] = 1
  else:
    prediction[i] = 0

accuracy = 0
for i in range(len(label)):
  if prediction[i] == label[i]:
    accuracy += 1
  
print("Accuracy: ", accuracy/len(label))

8/8 [==============================] - 0s 2ms/step
Accuracy:  0.876


In [16]:
import importlib
importlib.reload(test_lstm)

lie_test_path = './test/Deceptive/processed/'
label = 0

prediction_test = test_lstm.perdictSingleVideo(lie_test_path, MODEL, label, numOfFrames=10)

11/11 [==============================] - 0s 2ms/step
test.csv  accuracy:  0.7363896848137536  label:  0
39/39 [==============================] - 0s 1ms/step
test4.csv  accuracy:  0.33387227162489896  label:  0
35/35 [==============================] - 0s 1ms/step
test5.csv  accuracy:  0.09332113449222323  label:  0
34/34 [==============================] - 0s 1ms/step
test2.csv  accuracy:  0.48402255639097747  label:  0
23/23 [==============================] - 0s 1ms/step
test3.csv  accuracy:  0.4201909959072306  label:  0
17/17 [==============================] - 0s 2ms/step
test1.csv  accuracy:  0.005628517823639775  label:  0
